In [31]:
from env_hiv import *
from tqdm import tqdm
import os
import joblib
def setstate(env,state):
    env.T1 = state[0]
    env.T1star = state[1]
    env.T2 = state[2]
    env.T2star = state[3]
    env.V = state[4]
    env.E = state[5]
    return env
def beamsearch(env,state,action,depth, maxdepth) :
    env = setstate(env,state)
    state2, r,_,_,_ = env.step(action)
    possible_actions = [i for i in range(4)]
    if maxdepth == depth :
        return r

    else: 
        return r+max([beamsearch(env,state2,a,depth+1, maxdepth) for a in possible_actions])
def solve(env, maxdepth = 4) : 
    policy = []
    cumreward = 0
    state,_ = env.reset()
    possible_actions = [i for i in range(4)]
    for t in tqdm(range(200)):
        beam = joblib.Parallel(n_jobs=os.cpu_count())(joblib.delayed(beamsearch)(env, state, a,0, maxdepth) for a in possible_actions)
        action = np.argmax(beam)
        state , reward,_,_,_= env.step(action)
        cumreward+=reward
        policy.append(action)
    return policy,cumreward

In [ ]:
##VERSION 64 CPU
"""def beamsearchinit(env,state,action,depth,maxdepth):
    env = setstate(env,state)
    state2, r1,_,_,_ = env.step(action[0])
    state2, r2,_,_,_ = env.step(action[1])
    return r1+r2+beamsearch(env,state2,action[2],depth+2, maxdepth)
def solve(env, maxdepth = 4) : 
    policy = []
    cumreward = 0
    state,_ = env.reset()
    possible_actions = [[i,j,k] for i in range(4) for j in range(4) for k in range(4)]
    for t in tqdm(range(200)):
        beam = joblib.Parallel(n_jobs=os.cpu_count())(joblib.delayed(beamsearchinit)(env, state, a,0, maxdepth) for a in possible_actions)
        action = possible_actions[np.argmax(beam)][0]
        state , reward,_,_,_= env.step(action)
        cumreward+=reward
        policy.append(action)
    return policy,cumreward"""

##VERSION 128 CPU
"""def beamsearchinit(env,state,action,depth,maxdepth):
    env = setstate(env,state)
    state2, r1,_,_,_ = env.step(action[0])
    state2, r2,_,_,_ = env.step(action[1])
    state2, r3,_,_,_ = env.step(action[2])
    return r1+r2+r3+beamsearch(env,state2,action[2],depth+3, maxdepth)
def solve(env, maxdepth = 4) : 
    policy = []
    cumreward = 0
    state,_ = env.reset()
    possible_actions = [[i,j,k,l] for i in range(4) for j in range(4) for k in range(4) for l in range(4)]
    for t in tqdm(range(200)):
        beam = joblib.Parallel(n_jobs=os.cpu_count())(joblib.delayed(beamsearchinit)(env, state, a,0, maxdepth) for a in possible_actions)
        action = possible_actions[np.argmax(beam)][0]
        state , reward,_,_,_= env.step(action)
        cumreward+=reward
        policy.append(action)
    return policy,cumreward"""

In [49]:
from env_hiv import *
from tqdm import tqdm
import os
import joblib
def setstate(env,state):
    env.T1 = state[0]
    env.T1star = state[1]
    env.T2 = state[2]
    env.T2star = state[3]
    env.V = state[4]
    env.E = state[5]
    return env
def beamsearch(env,state,action,depth, maxdepth) :
    env = setstate(env,state)
    state2, r,_,_,_ = env.step(action)
    possible_actions = [i for i in range(4)]
    if maxdepth == depth :
        return r

    else: 
        return r+max([beamsearch(env,state2,a,depth+1, maxdepth) for a in possible_actions])

def beamsearchinit(env,state,action,depth,maxdepth):
    env = setstate(env,state)
    state2, r,_,_,_ = env.step(action[0])
    return r+beamsearch(env,state2,action[1],depth+1, maxdepth)
def solve(env, maxdepth = 4) : 
    policy = []
    cumreward = 0
    state,_ = env.reset()
    possible_actions = [[i,j] for i in range(4) for j in range(4)]
    for t in tqdm(range(200)):
        beam = joblib.Parallel(n_jobs=os.cpu_count())(joblib.delayed(beamsearchinit)(env, state, a,0, maxdepth) for a in possible_actions)
        action = possible_actions[np.argmax(beam)][0]
        setstate(env,state)
        state , reward,_,_,_= env.step(action)
        cumreward+=reward
        policy.append(action)
    return policy,cumreward

In [50]:
env = HIVPatient(domain_randomization=True)
pi,reward = solve(env)

100%|██████████| 200/200 [08:31<00:00,  2.56s/it]


In [51]:
"{:e}".format(reward)

'3.052717e+10'

In [ ]:
class ProjectAgent : 
    def __init__(self,pi) -> None:
        self.pi = pi
        self.cont = 0
    def act(self, obs, use_random =False) : 
        
        self.cont+=1
        return self.pi[self.cont-1]
agent = ProjectAgent(pi)
import random , torch , os
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)


from evaluate import *
seed_everything(seed=42)
score_agent: float = evaluate_HIV(agent=agent, nb_episode=1)
print('indiv ', {"{:e}".format(score_agent)})